# Clustering

Clustering algorithms are a subset of unsupervised machine learning techniques that group similar data points together based on certain characteristics or features. The main goal of clustering is to partition a dataset into groups, or clusters, where the data points within each cluster are more similar to each other than to those in other clusters.

There are various clustering algorithms, each with its own approach and characteristics. Some of the most common clustering algorithms include K-means, hierarchical clustering, DBSCAN (Density-Based Spatial Clustering of Applications with Noise), and Gaussian Mixture Models (GMM).

K-means is one of the simplest and most widely used clustering algorithms. It partitions the dataset into K clusters by iteratively assigning data points to the nearest cluster centroid and updating the centroids based on the mean of the data points assigned to each cluster.

Hierarchical clustering builds a tree-like hierarchy of clusters by either starting with individual data points as clusters and then merging them iteratively, or by starting with all data points as one cluster and then splitting them recursively.

DBSCAN identifies clusters based on the density of data points in the feature space. It groups together closely packed points and marks points in low-density regions as outliers.

Gaussian Mixture Models represent the distribution of data points as a mixture of multiple Gaussian distributions. It models each cluster as a Gaussian distribution and assigns data points probabilistically to each cluster.

Choosing the appropriate clustering algorithm depends on factors such as the structure of the data, the desired number of clusters, and the computational resources available. Evaluating clustering results often involves metrics such as silhouette score, Davies–Bouldin index, or visual inspection. Clustering finds applications in various fields including pattern recognition, image analysis, customer segmentation, and anomaly detection.

## Librays

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data

In [34]:
df = pd.read_csv("data\clients\marketing_campaign.csv", sep='\t')
print(df.columns)
df.head()

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [35]:
X = df[['Year_Birth', 'Income']]
X.head()

,Year_Birth,Income
0,1957,58138.0
1,1954,46344.0
2,1965,71613.0
3,1984,26646.0
4,1981,58293.0



## K-means


* K-means is an iterative procedure that
     * Starts by guessing the initial centroids, and then 
     * Refines this guess by 
         * Repeatedly assigning examples to their closest centroids, and then 
         * Recomputing the centroids based on the assignments.

* The inner-loop of the algorithm repeatedly carries out two steps: 
    1. Assigning each training example $x^{(i)}$ to its closest centroid, and
    2. Recomputing the mean of each centroid using the points assigned to it. 
    
    
* The $K$-means algorithm will always converge to some final set of means for the centroids. 

* However, the converged solution may not always be ideal and depends on the initial setting of the centroids.
    * Therefore, in practice the K-means algorithm is usually run a few times with different random initializations. 
    * One way to choose between these different solutions from different random initializations is to choose the one with the lowest cost function value (distortion).

In [36]:
def find_closest_centroids(X, centroids):
    """
    Computes the centroid memberships for every example
    
    Args:
        X (ndarray): (m, n) Input values      
        centroids (ndarray): (K, n) centroids
    
    Returns:
        idx (array_like): (m,) closest centroids
    
    """
    
    K = centroids.shape[0]
    idx = np.zeros(X.shape[0], dtype=int)
    
    for i in range(X.shape[0]):
        distance = [] 
        for j in range(centroids.shape[0]):
            # Compute Euclidean distance
            norm_ij = np.linalg.norm(X[i] - centroids[j])
            distance.append(norm_ij)
        idx[i] = np.argmin(distance)
    
    return idx

For every centroid $\mu_k$, is seted

$$\mu_k = \frac{1}{|C_k|} \sum_{i \in C_k} x^{(i)}$$ 

    where 
    * $C_k$ is the set of examples that are assigned to centroid $k$
    * $|C_k|$ is the number of examples in the set $C_k$


If two examples say $x^{(3)}$ and $x^{(5)}$ are assigned to centroid $k=2$,
then you should update $\mu_2 = \frac{1}{2}(x^{(3)}+x^{(5)})$.

In [37]:
def compute_centroids(X, idx, K):
    """
    Returns the new centroids by computing the means of the 
    data points assigned to each centroid.
    
    Args:
        X (ndarray):   (m, n) Data points
        idx (ndarray): (m,) Array containing index of closest centroid for each 
                       example in X. Concretely, idx[i] contains the index of 
                       the centroid closest to example i
        K (int):       number of centroids
    
    Returns:
        centroids (ndarray): (K, n) New centroids computed
    """
    
    m, n = X.shape
    centroids = np.zeros((K, n))
    
    for k in range(K):
        X_k = X[idx == k]
        c_k = np.mean(X_k, axis=0)
        
        # Alternative:
        # X_k= []
        # for i, id in enumerate(idx):
        #     if id == k:
        #         X_k2.append(X[i])

        # c_k = sum(X_k2) / len(X_k2)
        
        centroids[k] = c_k
    
    return centroids

In [41]:
def kMeans_init_centroids(X, K):
    """
    This function initializes K centroids that are to be 
    used in K-Means on the dataset X
    
    Args:
        X (ndarray): Data points 
        K (int):     number of centroids/clusters
    
    Returns:
        centroids (ndarray): Initialized centroids
    """
    
    # Randomly reorder the indices of examples
    randidx = np.random.permutation(X.shape[0])
    
    # Take the first K examples as centroids
    centroids = X[randidx[:K]]
    
    return centroids

## Run kmeans

In [38]:
def plot_progress_kMeans(X, centroids, previous_centroids, idx, K, i):
    """
    Plots the progress of the K-Means algorithm
    """
    plt.scatter(X[:, 0], X[:, 1], c=idx, cmap='viridis', marker='.')
    plt.scatter(centroids[:, 0], centroids[:, 1], marker='o', s=300, c='r', edgecolors='k', label='Centroids')
    plt.scatter(previous_centroids[:, 0], previous_centroids[:, 1], marker='x', s=200, c='k', label='Previous Centroids')
    plt.title(f"Iteration {i+1}")
    plt.legend()
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.grid(True)

In [39]:
def run_kMeans(X, initial_centroids, max_iters=10, plot_progress=False):
    """
    Runs the K-Means algorithm on data matrix X, where each row of X
    is a single example
    """
    
    X = X.values
    # Initialize values
    m, n = X.shape
    K = initial_centroids.shape[0]
    centroids = initial_centroids
    previous_centroids = centroids    
    idx = np.zeros(m)
    plt.figure(figsize=(8, 6))
    
    # Run K-Means
    for i in range(max_iters):
        
        #Output progress
        print("K-Means iteration %d/%d" % (i+1, max_iters))
        
        # For each example in X, assign it to the closest centroid
        idx = find_closest_centroids(X, centroids)
        
        # Optionally plot progress
        if plot_progress:
            plot_progress_kMeans(X, centroids, previous_centroids, idx, K, i)
            previous_centroids = centroids.copy()
        
        # Given the memberships, compute new centroids
        centroids = compute_centroids(X, idx, K)
    
    plt.show() 
    return centroids, idx